In [1874]:
from Imports import edge_detection
from Imports import hough_transform
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.metrics import accuracy_score
import joblib


# Load Coin Data
data = np.load("Data/data2.npy")
print(data.shape)

X = data[:, :-1]
y = data[:, -1]

# Perform Train_Test_Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


(158, 257)


In [1875]:
# Perform Normalization
mean_vec = np.mean(X_train, axis=0)
std_vec = np.std(X_train, axis=0)
print(mean_vec.shape, std_vec.shape)

normalized_X = (X_train - mean_vec) / (std_vec + 1e-15)
print(X_train.shape)

(256,) (256,)
(118, 256)


In [1876]:
# Perform Feature Selection
sel = VarianceThreshold(threshold=1e-1)
selector1 = sel.fit(normalized_X)
X_new = selector1.transform(normalized_X)
selector2 = SelectKBest(f_classif, k=20).fit(X_new, y_train)
X_new = selector2.transform(X_new)
print(X_new.shape)

(118, 20)


C:\Users\jason\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_selection\_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


In [1877]:
# Fit model
model = SVC(C=100, kernel="rbf")
model.fit(X_new, y_train)

# Preprocess
normalized_X_test = (X_test - mean_vec) / (std_vec + 1e-13)
X_test_new = selector1.transform(normalized_X_test)
X_test_new = selector2.transform(X_test_new)
ypred = model.predict(X_test_new)

print("The accuracy is: ", accuracy_score(y_test, ypred))
print(ypred.shape)

The accuracy is:  0.95
(40,)


In [1878]:
#Finally, train on all data and save enitre pipeline as model

# Create a pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selector_var', VarianceThreshold(threshold=1e-1)),
    ('feature_selector_kbest', SelectKBest(f_classif, k=20)),
    ('classifier', SVC(C=100, kernel="rbf"))
])

# Train the pipeline
pipe.fit(X, y)

joblib.dump(pipe, 'Models/svc.pkl')


C:\Users\jason\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\jason\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\jason\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\jason\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_selection\_variance_t

['Models/svc.pkl']